<a href="https://colab.research.google.com/github/Izahmed35/TEST1/blob/main/E_bookauthor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-generativeai reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.6 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from getpass import getpass
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER
import os
import time
import random
import re

# Securely input your API key
api_key = getpass("Enter your Gemini API key: ")

# Set up your API key
genai.configure(api_key=api_key)

# Initialize the model
model = genai.GenerativeModel('gemini-1.5-pro')

def generate_content(prompt, max_attempts=3):
    for attempt in range(max_attempts):
        try:
            response = model.generate_content(prompt)
            if response.text:
                return response.text
            else:
                print(f"Warning: Empty response on attempt {attempt + 1}.")
        except Exception as e:
            print(f"Error on attempt {attempt + 1}: {str(e)}")
            if attempt == max_attempts - 1:
                raise
        time.sleep(random.uniform(1, 3))
    return "Failed to generate content after multiple attempts."

def generate_book_title(topic, style):
    prompt = f"""Generate a single, poetic and engaging title for a book about {topic} written in the style of {style}.
    The title should be evocative, relevant to the topic and style, and capture the essence of a love story set against the backdrop of war.
    Provide only the title, without any additional explanation."""
    return generate_content(prompt)

def generate_book_outline(topic, style, num_chapters):
    prompt = f"""Create a detailed outline for a {num_chapters}-chapter book about {topic} in the style of {style}.
    The outline should include:
    1. A brief introduction to the book's theme
    2. For each chapter:
       a. A poetic chapter title
       b. A brief synopsis (2-3 sentences)
       c. Key plot points or concepts to be covered
       d. Character development or thematic elements to focus on
    Use rich, poetic language throughout the outline."""
    return generate_content(prompt)

def generate_chapter(topic, style, chapter_number, chapter_outline):
    prompt = f"""Write chapter {chapter_number} of a book about {topic} in the style of {style}.
    Use the following outline as a guide:

    {chapter_outline}

    The chapter should be approximately 1500-2000 words long, rich in poetic language, vivid imagery, and emotional depth.
    Focus on creating a powerful atmosphere that captures the tension between love and war.
    Include dialogue, descriptive passages, and character thoughts as appropriate for the style and topic."""
    return generate_content(prompt)

def sanitize_filename(filename):
    sanitized = re.sub(r'[<>:"/\\|?*]', '', filename)
    return sanitized[:200]  # Limit to 200 characters

def write_book():
    topic = input("Enter the book topic or description: ")
    style = input("Enter the writing style (e.g., 'poetic', 'lyrical'): ")
    num_chapters = int(input("Enter the number of chapters for the book: "))

    book_title = generate_book_title(topic, style)
    print(f"Generated book title: {book_title}")

    print("Generating book outline...")
    book_outline = generate_book_outline(topic, style, num_chapters)
    print("Book outline generated.")

    chapters = []
    for chapter in range(1, num_chapters + 1):
        print(f"Generating chapter {chapter}...")
        chapter_outline = extract_chapter_outline(book_outline, chapter)
        chapter_content = generate_chapter(topic, style, chapter, chapter_outline)
        chapters.append(chapter_content)
        time.sleep(random.uniform(2, 5))

    return book_title, book_outline, chapters

def extract_chapter_outline(book_outline, chapter_number):
    # This function should extract the relevant chapter outline from the full book outline
    # You may need to adjust this based on the exact format of your generated outline
    chapter_start = book_outline.find(f"Chapter {chapter_number}")
    if chapter_start == -1:
        return "Chapter outline not found."
    next_chapter = book_outline.find(f"Chapter {chapter_number + 1}", chapter_start)
    if next_chapter == -1:
        return book_outline[chapter_start:]
    return book_outline[chapter_start:next_chapter]

def save_book_as_pdf(book_title, book_outline, chapters, filename=None):
    if filename is None:
        filename = sanitize_filename(book_title) + ".pdf"

    doc = SimpleDocTemplate(filename, pagesize=letter)
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    styles.add(ParagraphStyle(name='Center', alignment=TA_CENTER))

    flowables = []

    # Add title page
    flowables.append(Paragraph(book_title, styles['Title']))
    flowables.append(PageBreak())

    # Add book outline
    flowables.append(Paragraph("Book Outline", styles['Heading1']))
    flowables.append(Paragraph(book_outline, styles['BodyText']))
    flowables.append(PageBreak())

    # Add table of contents
    flowables.append(Paragraph("Table of Contents", styles['Heading1']))
    for i in range(len(chapters)):
        flowables.append(Paragraph(f"Chapter {i+1}", styles['Normal']))
    flowables.append(PageBreak())

    # Add chapters
    for i, chapter in enumerate(chapters, 1):
        flowables.append(Paragraph(f"Chapter {i}", styles['Heading1']))
        flowables.append(Spacer(1, 12))

        paragraphs = chapter.split('\n')
        for para in paragraphs:
            if para.strip():
                flowables.append(Paragraph(para, styles['Justify']))
                flowables.append(Spacer(1, 6))

        flowables.append(PageBreak())

    doc.build(flowables)
    print(f"Book saved as {filename}")

if __name__ == "__main__":
    print("Welcome to the Improved AI Book Writing Tool using Gemini 1.5 Pro!")
    try:
        book_title, book_outline, chapters = write_book()
        save_book_as_pdf(book_title, book_outline, chapters)
        print("Book generation complete!")
    except Exception as e:
        print(f"An error occurred during book generation: {str(e)}")

Enter your Gemini API key: ··········
Welcome to the Improved AI Book Writing Tool using Gemini 1.5 Pro!
Enter the book topic or description: The story of Hitler
Enter the writing style (e.g., 'poetic', 'lyrical'): Historical
Enter the number of chapters for the book: 1
Generated book title: The Führer's Heartbreak  

Generating book outline...
Book outline generated.
Generating chapter 1...
Book saved as The Führer's Heartbreak  
.pdf
Book generation complete!


In [ ]:
!pip install arabic-reshaper python-bidi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from getpass import getpass
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER, TA_RIGHT
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import requests
import io
from bidi.algorithm import get_display
import arabic_reshaper
import os
import time
import random
import re

# تحميل وتسجيل خط عربي من Google Fonts API
def download_google_font(font_name):
    url = f"https://fonts.googleapis.com/css?family={font_name}"
    response = requests.get(url)
    font_url = response.text.split("url(")[1].split(")")[0]
    font_response = requests.get(font_url)
    return io.BytesIO(font_response.content)

arabic_font = 'Cairo'  # يمكنك تغيير هذا إلى أي خط عربي آخر متاح في Google Fonts
font = download_google_font(arabic_font)
pdfmetrics.registerFont(TTFont(arabic_font, font))

# إعداد Gemini API
api_key = getpass("أدخل مفتاح API الخاص بـ Gemini: ")
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-1.5-pro')

def generate_content(prompt, max_attempts=3):
    for attempt in range(max_attempts):
        try:
            response = model.generate_content(prompt)
            if response.text:
                return response.text
            else:
                print(f"تحذير: استجابة فارغة في المحاولة {attempt + 1}.")
        except Exception as e:
            print(f"خطأ في المحاولة {attempt + 1}: {str(e)}")
            if attempt == max_attempts - 1:
                raise
        time.sleep(random.uniform(1, 3))
    return "فشل في إنشاء المحتوى بعد عدة محاولات."

def generate_book_title(topic, style):
    prompt = f"قم بإنشاء عنوان واحد شعري وجذاب لكتاب عن {topic} بأسلوب {style} باللغة العربية. قدم العنوان فقط دون أي شروحات إضافية."
    return generate_content(prompt).strip()

def generate_book_outline(topic, style, num_chapters):
    prompt = f"""قم بإنشاء مخطط تفصيلي لكتاب من {num_chapters} فصلاً عن {topic} بأسلوب {style} باللغة العربية.
    لكل فصل، قدم عنواناً وملخصاً موجزاً (2-3 جمل).
    قدم المخطط بالتنسيق التالي:
    الفصل 1: [عنوان الفصل]
    [ملخص الفصل]

    الفصل 2: [عنوان الفصل]
    [ملخص الفصل]

    وهكذا..."""
    return generate_content(prompt)

def generate_chapter(topic, style, chapter_number, chapter_outline):
    prompt = f"""اكتب الفصل {chapter_number} من كتاب عن {topic} بأسلوب {style} باللغة العربية.
    استخدم هذا المخطط كدليل:
    {chapter_outline}

    اكتب محتوى الفصل بحيث يكون حوالي 1000 كلمة."""
    return generate_content(prompt)

def write_book():
    topic = input("أدخل موضوع الكتاب: ")
    style = input("أدخل الأسلوب الكتابي: ")
    num_chapters = int(input("أدخل عدد الفصول: "))

    book_title = generate_book_title(topic, style)
    print(f"عنوان الكتاب: {book_title}")

    book_outline = generate_book_outline(topic, style, num_chapters)
    print("تم إنشاء مخطط الكتاب.")

    chapters = []
    for chapter in range(1, num_chapters + 1):
        print(f"جاري إنشاء الفصل {chapter}...")
        chapter_outline = extract_chapter_outline(book_outline, chapter)
        chapter_content = generate_chapter(topic, style, chapter, chapter_outline)
        chapters.append(chapter_content)

    return book_title, book_outline, chapters

def extract_chapter_outline(book_outline, chapter_number):
    chapters = book_outline.split(f"الفصل {chapter_number}:")
    if len(chapters) > 1:
        return chapters[1].split(f"الفصل {chapter_number + 1}:")[0].strip()
    return "لم يتم العثور على مخطط الفصل."

def process_arabic_text(text):
    reshaped_text = arabic_reshaper.reshape(text)
    return get_display(reshaped_text)

def save_book_as_pdf(book_title, book_outline, chapters):
    filename = f"{book_title}.pdf"
    filename = re.sub(r'[<>:"/\\|?*]', '', filename)
    filename = filename[:200] + ".pdf"

    doc = SimpleDocTemplate(filename, pagesize=A4, encoding='utf-8', rightMargin=72, leftMargin=72, topMargin=72, bottomMargin=18)
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Arabic', fontName=arabic_font, fontSize=12, alignment=TA_RIGHT, leading=14))

    story = []

    # صفحة العنوان
    story.append(Paragraph(process_arabic_text(book_title), styles['Title']))
    story.append(PageBreak())

    # المخطط
    story.append(Paragraph(process_arabic_text("مخطط الكتاب"), styles['Heading1']))
    story.append(Paragraph(process_arabic_text(book_outline), styles['Arabic']))
    story.append(PageBreak())

    # الفصول
    for i, chapter in enumerate(chapters, 1):
        story.append(Paragraph(process_arabic_text(f"الفصل {i}"), styles['Heading1']))
        paragraphs = chapter.split('\n')
        for para in paragraphs:
            if para.strip():
                story.append(Paragraph(process_arabic_text(para), styles['Arabic']))
        story.append(PageBreak())

    doc.build(story)
    print(f"تم حفظ الكتاب باسم {filename}")

if __name__ == "__main__":
    print("مرحباً بك في أداة إنشاء الكتب العربية!")
    try:
        book_title, book_outline, chapters = write_book()
        save_book_as_pdf(book_title, book_outline, chapters)
        print("اكتمل إنشاء الكتاب!")
    except Exception as e:
        print(f"حدث خطأ: {str(e)}")
        print("يرجى المحاولة مرة أخرى أو التحقق من اتصال الإنترنت ومفتاح API.")

مرحباً بك في أداة إنشاء الكتب العربية!


KeyboardInterrupt: Interrupted by user